In [1]:
import cv2
import matplotlib.pyplot as plt
import skimage.segmentation as seg
import skimage.filters as filt
import skimage.morphology as morph
import skimage.draw as draw
from scipy import ndimage
from mpl_toolkits.mplot3d import Axes3D
from scipy.integrate import cumtrapz
import scipy.ndimage.filters as filters
import os
import numpy as np
import pandas as pd
from plantcv import plantcv as pcv 

d:\anaconda3\envs\tf2.4\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
d:\anaconda3\envs\tf2.4\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
d:\anaconda3\envs\tf2.4\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"
<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [2]:
class options:
    def __init__(self):
        self.debug = "plot"
        self.writeimg= 'False' 
        self.result = "features_metadata.json"
        self.outdir = ""
# Get options
args = options()

# Set debug to the global parameter 
pcv.params.debug = args.debug
pcv.params.debug = args.debug

In [4]:
def curvature(x, y, xc, yc, r):
    # Shift coordinates to make the center of the circle the origin
    x_shifted = x - xc
    y_shifted = y - yc
    # Calculate distance from each point to the center of the circle
    d = np.sqrt(x_shifted**2 + y_shifted**2)
    # Calculate the curvature only for points inside the circle
    inside_circle = d <= r
    dx_dt = np.gradient(x[inside_circle])
    dy_dt = np.gradient(y[inside_circle])
    d2x_dt2 = np.gradient(dx_dt)
    d2y_dt2 = np.gradient(dy_dt)
    curvature = (dx_dt * d2y_dt2 - d2x_dt2 * dy_dt) / (dx_dt**2 + dy_dt**2)**(3/2)
    # Pad the curvature array with zeros for the points outside the circle
    curvature_padded = np.zeros_like(d)
    curvature_padded[inside_circle] = curvature
    return curvature_padded


def integral_curvature(x, y, xc, yc, r):
    k = curvature(x, y, xc, yc, r)
    return cumtrapz(k, initial=0)


def get_integral_curvature(i,x,y, r):
    # Calculate the integral curvature for a circle centered at (x[i], y[i]) with radius r
    int_curv = integral_curvature(x, y, x[i], y[i], r)[-1]
    return int_curv


def get_area_measure(contour,i,radius, thresh):
    # Approximate the contour with a circle centered at the current point
    circle_mask = np.zeros_like(gray)
    cv2.circle(circle_mask, (contour[i][0][0], contour[i][0][1]), radius, (255, 255, 255), -1)
    intersection_mask = cv2.bitwise_and(circle_mask, thresh)
    intersection_area = np.sum(intersection_mask) / 255
    circle_area = np.pi * radius ** 2
    intersection_fraction = intersection_area / circle_area
    # Compute the curvature value for the current point
    return intersection_fraction


def get_curvature_arc_length(contour,i,x,y,radius,circle_perimeters):
    curvature_arr = []
    # Approximate the contour with a circle centered at the current point
    (cx, cy) = x[i], y[i]
    # Calculate the length of the part of the circle's circumference that is inside the object
    chord_length = 2 * np.sqrt(radius**2 - ((radius**2-(x[i]-cx)**2-(y[i]-cy)**2))/4)
    arc_length = 2 * np.arcsin(chord_length / (2 * radius))
    circle_perimeters.append(arc_length)
    if len(circle_perimeters) > 0:
        # Calculate the average perimeter of the fitted circles
        circle_perimeter = np.mean(circle_perimeters)
    else:
        # Use the radius as an estimate for the perimeter
        circle_perimeter = 2 * np.pi * radius

    # Calculate perimeter of the contour
    contour_perimeter = cv2.arcLength(contour, True)
    # Calculate the fraction of the circle's perimeter contained inside the object
    fraction = circle_perimeter / contour_perimeter
    arc_len = fraction * arc_length
    return arc_len


def get_curvatures(contours, radius_arr,thresh):
    curvature_arr=[]
    # Loop over each contour
    for contour in contours:
        curr = []
        # Precompute the coordinates of all points in the contour
        x, y = contour[:, 0, 0], contour[:, 0, 1]
        # Compute the maximum and minimum x and y coordinates of the contour
        min_x, max_x = np.min(x), np.max(x)
        min_y, max_y = np.min(y), np.max(y)
        # Compute the thresholded image for the current contour
        mask = np.zeros_like(gray)
        cv2.drawContours(mask, [contour], 0, (255, 255, 255), -1)
        thresh = cv2.bitwise_and(gray, mask)
        # Compute the area measure for each point in the contour
        for radius in radius_arr:
            circle_perimeters = []
            arc_length = 0  # Initialize arc_length to 0
            curr = []
            for i in range(len(contour)):
                curr_area = get_area_measure(contour,i,radius,thresh)
                # curr_arc = get_curvature_arc_length(contour,i,x,y,radius,circle_perimeters)
                # curr_int = get_integral_curvature(i,x,y, radius)
                curr.append(curr_area) # remove
                #curr.append(curr_arc)
                #curr.append(curr_int) # add
            curvature_arr.append(curr)

    return curvature_arr
            #print( f'area={curr_area} arc ={curr_arc} curr int = {curr_int}')

In [26]:
# # Convert RGB to HSV and extract the saturation channel
# s = pcv.rgb2gray_hsv(rgb_img=img, channel='s')
# # Threshold the saturation image
# s_thresh = pcv.threshold.binary(gray_img=s, threshold=85, max_value=255, object_type='light')


# specify path to the folder containing image files
path = "../../leafsnap/leafsnap-dataset/dataset/segmented/field/"

counter = 0

# initialize empty lists for features and labels
features = []
labels = []

# loop through all subfolders in the path
for foldername in os.listdir(path):
    if counter == 1:
        break
    
    folderpath = os.path.join(path, foldername)
    if not os.path.isdir(folderpath):
        continue
        
    # loop through all image files in the subfolder
    for filename in os.listdir(folderpath):
        filepath = os.path.join(folderpath, filename)
        if not os.path.isfile(filepath):
            continue
        
        # try:
        # load the image file and extract features
        ini_img = cv2.imread(filepath)

        # resize the image
        img = cv2.resize(ini_img, (250, 250))

        # Convert the image to grayscale
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        # test = pcv.threshold.binary(gray_img=gray, threshold=85, max_value=255, object_type='dark')
        # Threshold the image to get a binary image
        ret, thresh = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY)
        # Find the contours in the image
        contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        radius_arr = [10,20,30,40,50,60,70,80,90,100,110,120,130,140,150,160,170,180,190,200,210,220,230,240,250]
        icurvature_arr = get_curvatures(contours, radius_arr, thresh)
        curvature_arr = np.array(icurvature_arr)
        print(icurvature_arr)
        x = np.arange(curvature_arr.shape[1]) # x-axis is the number of contour points
        y = np.arange(curvature_arr.shape[0]) # y-axis is the different scales
        X, Y = np.meshgrid(x, y)
        hist_values = []
        for i in range(curvature_arr.shape[0]):
            counts, bin_edges = np.histogram(curvature_arr[i], bins=30) # adjust the number of bins as needed
            hist_values.append(counts)
        data = np.array([np.array(hist) for hist in hist_values])
        X = data.flatten().tolist()
        
        features.append(X)
        labels.append(foldername)
        # except:
        #     pass
            
    print(f"Finished plant: {foldername}")
    counter += 1


# convert features and labels to numpy arrays
features = features
labels = np.array(labels)

# save features and labels to a pandas dataframe and export to CSV file
data = pd.DataFrame({"hist_values" : features})
data["plant"] = labels
data.to_csv("leafsnap_data_fin.csv", index=False)

[[0.47022483970758405, 0.49603914734005783, 0.4706991838517011, 0.49298087588456646, 0.47910006829882623, 0.49822362695112304, 0.4792748266677115, 0.4983734198387389, 0.47722765720362753, 0.4950405280892851, 0.47842600030455473, 0.49847328176381617, 0.48363130314920727, 0.5056633403693794, 0.48384350973999646, 0.5038783084586234, 0.47926234392707684, 0.49764942088192876, 0.4734578695319607, 0.48755088370849, 0.43195275692172125, 0.4287696580598833, 0.3525625264852935, 0.3302808344524281, 0.3031558390533153, 0.27921394251604037, 0.22617477755937662, 0.17747960634357396, 0.17429650748173606, 0.19505530515717306, 0.2957660565975975, 0.31181886105376827, 0.4498654897324562, 0.474905867445581, 0.47557993543985255, 0.4971501112565424, 0.473407938569422, 0.4956521823803834, 0.48008620480896425, 0.4991847979799917, 0.4844426812904601, 0.49814873050731506, 0.4847048188437879, 0.5013068638878837, 0.48180882301654715, 0.4981362477666804, 0.4792748266677115, 0.4983734198387389, 0.4852665421723475,

C:\Users\Jeremy\AppData\Local\Temp\ipykernel_7172\2941244644.py:47: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


IndexError: tuple index out of range

In [25]:
test = [[0.0015977908012362826], [0.00039944770030907065, 0.9238728382382], [0.00017753231124847585], [9.986192507726766e-05]]

arr = np.concatenate(test).reshape(-1, len(test))
arr

ValueError: cannot reshape array of size 5 into shape (4)